<a href="https://colab.research.google.com/github/dkimds/goormthon/blob/main/%08clf_naver_news_sub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 뉴스 카테고리 분류기 만들기

## 자연어처리 라이브러리 설치

In [ ]:
# nltk.download()  # 텍스트 데이터 처리를 위한 패키지 다운로더

# Download following packages
# Corpora : stopwords, wordnet
# Models : averaged_perceptron_tagger, maxent_treebank_pos_tagger, punkt
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> stopwords


      Unzipping corpora/stopwords.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> wordnet



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt


      Unzipping tokenizers/punkt.zip.


...

        /root/nltk_data...
      Unzipping taggers/averaged_perceptron_tagger.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> maxent_treebank_pos_tagger


        /root/nltk_data...
      Unzipping taggers/maxent_treebank_pos_tagger.zip.



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [ ]:
# Install konlpy
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2023-12-12 23:05:16--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2023-12-12 23:05:16--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc635f87bb008720fe23a3b2bf29.dl.dropboxusercontent.com/cd/0/get/CJRrCP3GPiieSjFGv1g0LaOshRFMYqrskj3OZ_SLMdKTFFKaU2FBHz2KV0OeluLJocUYhCCUBSrCT-nutuxE5GqKJletxNhlqDNAI8REUUlNwLUht2zhRRtOPse6tix1tdZXZDcNF6I4EB61HKm88DpU/file?dl=1# [following]
--2023-12-12 23:05:17--  https://uc635f87bb008720fe23a3b2bf29.dl.dropboxusercontent.com/cd/0/get/CJRrCP3GPiieSjFGv1g0LaOshRFMYqrskj3OZ_SLMdKTFFKaU2FBHz2

## 데이터셋 불러오기

In [ ]:
!ls

clf_news_MultinomialNB_v1.pkl  result_231210_0127.csv  result_231210_0221.csv  sample_data
result_231210_0111.csv	       result_231210_0206.csv  result_231210_0239.csv


In [ ]:
import pandas as pd
import numpy as np

csv_list = ['result_231210_0111.csv',
            'result_231210_0206.csv',
            'result_231210_0239.csv',
            'result_231210_0127.csv',
            'result_231210_0221.csv']
news_df = pd.DataFrame()
for csv in csv_list:
    df_data = pd.read_csv(csv)
    news_df = news_df.append(df_data)

print('전체 리뷰 수:', len(news_df))
news_df.head()

전체 리뷰 수: 2316


<ipython-input-3-9f634c9de91f>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news_df.append(df_data)
<ipython-input-3-9f634c9de91f>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news_df.append(df_data)
<ipython-input-3-9f634c9de91f>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news_df.append(df_data)
<ipython-input-3-9f634c9de91f>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news_df.append(df_data)
<ipython-input-3-9f634c9de91f>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  news_df = news

,date,category,press,title,document,link
0,2023.11.29. 오후 12:00,사회,연합뉴스,3분기 출산율 0.7명 '역대 최저'…0.6명대 추락 '빨간불',"통계청, 9월 인구동향…작년 3분기比 0.1명↓47개월째 인구 자연감소(세종=연합뉴...",https://n.news.naver.com/mnews/article/001/001...
1,2023.11.29. 오후 6:04,경제,한국경제,통계나올 때마다 '쇼크'…3분기 출산율 0.7명 또 역대 최저,통계청 인구동향17개 광역시·도 일제히 하락광주 0.2명 줄어 하락폭 최대 인구 4...,https://n.news.naver.com/mnews/article/015/000...
2,2023.11.30. 오전 5:03,사회,서울신문,3분기도 출산율 최저… 올 0.7명도 위태롭다,1년 전보다 0.1명 줄어들어 0.7명9월 출생아 수·혼인도 가장 낮아다문화 혼인은...,https://n.news.naver.com/mnews/article/081/000...
3,2023.11.29. 오후 3:29,경제,한경비즈니스,“이러다 한국 진짜 망한다”...또 다시 ‘출산율 쇼크’,3분기 합계출산율 0.7명 기록3분기 기준 역대 최저치4분기에는 더 떨어질 가능성 ...,https://n.news.naver.com/mnews/article/050/000...
4,2023.11.29. 오후 9:33,경제,TV조선,"3분기 합계 출산율, 0.7명…올해 출생아 20만 명대 턱걸이","[앵커]출산율 감소, 혼인율 감소 저도 계속 전하는 입장에서 뾰족한 대책이 없...",https://n.news.naver.com/mnews/article/448/000...


In [ ]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2316 entries, 0 to 469
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      2316 non-null   object
 1   category  2316 non-null   object
 2   press     2316 non-null   object
 3   title     2316 non-null   object
 4   document  2307 non-null   object
 5   link      2316 non-null   object
dtypes: object(6)
memory usage: 126.7+ KB


In [ ]:
news_df = news_df.dropna()

In [ ]:
set(news_df['category'])

{'IT', '경제', '사회', '생활', '세계', '오피니언', '정치'}

In [ ]:
import re

def tokenize_and_stemming(sentence, tokenizer):

    sentence = re.sub("[^\s0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣]", "", sentence)

    raw_pos_tagged = tokenizer.pos(sentence, stem=True) # POS Tagging with stemming

    sentence_tokenized = []
    for token, pos in raw_pos_tagged: # ex) word <- ('서울', 'Noun')
        # pos 기준 제거 대상 : ["Josa", "Eomi", "Punctuation", "Foreign", "Number", "Suffix", "Determiner"]
        # 리뷰 텍스트 상 형태소의 순서를 고려하는 경우 의미를 갖게 됨 : "VerbPrefix" (잘/안/못), "Conjunction"(하지만/그러나/그리고), "KoreanParticle"(ㅋㅋㅋ/ㅎㅎㅎ/ㅠㅠ)
        if (len(token) != 1) & (pos in ["Noun", "VerbPrefix", "Verb", "Adverb", "Adjective", "Conjunction", "KoreanParticle"]):
            sentence_tokenized.append(token)

    return sentence_tokenized

In [ ]:
from konlpy.tag import Okt
import tqdm

okt = Okt()
tokenized_sentences = []

for sentence in tqdm.tqdm(news_df['document']):
    try:
        tokenized_sentences.append(tokenize_and_stemming(sentence, okt))
    except:
        print("Error occured at :", sentence)
        tokenized_sentences.append([])

news_df['document'] = tokenized_sentences

100%|██████████| 2307/2307 [04:32<00:00,  8.47it/s]


In [ ]:
# https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
categories = news_df['category'].tolist()
le.fit(categories)
news_df['category'] = le.transform(categories)

In [ ]:
news_df.head()

,date,category,press,title,document,link
0,2023.11.29. 오후 12:00,2,연합뉴스,3분기 출산율 0.7명 '역대 최저'…0.6명대 추락 '빨간불',"[통계청, 인구, 동향, 작년, 개월, 인구, 자연, 감소, 세종, 연합뉴스, 이준...",https://n.news.naver.com/mnews/article/001/001...
1,2023.11.29. 오후 6:04,1,한국경제,통계나올 때마다 '쇼크'…3분기 출산율 0.7명 또 역대 최저,"[통계청, 인구, 동향, 광역시, 일제, 하락, 광주, 줄다, 하락, 최대, 인구,...",https://n.news.naver.com/mnews/article/015/000...
2,2023.11.30. 오전 5:03,2,서울신문,3분기도 출산율 최저… 올 0.7명도 위태롭다,"[전보, 줄어들다, 출생, 혼인, 가장, 낮다, 다문화, 혼인, 늘다, 올해, 합계...",https://n.news.naver.com/mnews/article/081/000...
3,2023.11.29. 오후 3:29,1,한경비즈니스,“이러다 한국 진짜 망한다”...또 다시 ‘출산율 쇼크’,"[합계출산율, 기록, 기준, 역대, 최저, 떨어지다, 가능성, 높다, 통계청, 발표...",https://n.news.naver.com/mnews/article/050/000...
4,2023.11.29. 오후 9:33,1,TV조선,"3분기 합계 출산율, 0.7명…올해 출생아 20만 명대 턱걸이","[앵커, 출산율, 감소, 혼인, 감소, 계속, 전하, 입장, 뾰족, 대책, 없다, ...",https://n.news.naver.com/mnews/article/448/000...


## 트레이닝/테스트 셋으로 나누기

In [ ]:
from sklearn import model_selection

train_x, test_x, train_y, test_y = model_selection.train_test_split(news_df['document'], news_df['category'],
                                                                    test_size=0.3,
                                                                    random_state=42)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer # CountVectorizer

def list_to_list(doc):
    return doc

vectorizer = TfidfVectorizer(tokenizer=list_to_list, token_pattern=None, lowercase=False) # CountVectorizer 적용 시 아주 약간 성능 저하 (88.15% -> 88.02%)
vectorizer.fit(train_x)

train_x_vector = vectorizer.transform(train_x) # .toarray() 함수를 연달아 적용하면 np.array 형태로 만들어진 행렬을 확인할 수 있으나, RAM이 초과되어 Colab이 초기화됩니다.
test_x_vector = vectorizer.transform(test_x)

## 모델 생성

In [ ]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB
mnb_clf = MultinomialNB(force_alpha=True)
cnb_clf = ComplementNB(force_alpha=True)

## 모델 학습

In [ ]:
import numpy as np

x = train_x_vector.toarray()

mnb_clf.fit(train_x_vector, train_y)

MultinomialNB(force_alpha=True)

In [ ]:
cnb_clf.fit(train_x_vector, train_y)

ComplementNB(force_alpha=True)

## 테스트 데이터 예측

In [ ]:
mnb_pred_y = mnb_clf.predict(test_x_vector)
cnb_pred_y = cnb_clf.predict(test_x_vector)

# 모델 평가

## Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score
mnb_mean = sum(cross_val_score(mnb_clf, train_x_vector, train_y, cv=5))/5
print(mnb_mean)

0.6555006441936042


In [ ]:
cnb_mean = sum(cross_val_score(cnb_clf, train_x_vector, train_y, cv=5))/5
print(cnb_mean)

0.7044535892160068


## Accuracy score

In [ ]:
cnb_clf.score(test_x_vector, test_y)

from sklearn.metrics import accuracy_score

accuracy_score(test_y, cnb_pred_y)

0.7171717171717171

## Classification Report

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_y, cnb_pred_y))

              precision    recall  f1-score   support

           0       0.61      0.46      0.52        50
           1       0.73      0.82      0.77       215
           2       0.81      0.69      0.74       220
           3       0.67      0.86      0.76       109
           4       0.58      0.79      0.67        58
           5       0.00      0.00      0.00        23
           6       1.00      0.33      0.50        18

    accuracy                           0.72       693
   macro avg       0.63      0.57      0.57       693
weighted avg       0.71      0.72      0.70       693



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(test_y, cnb_pred_y))

[[ 23   7   4   0  16   0   0]
 [  8 177   9   8  13   0   0]
 [  2  27 151  38   2   0   0]
 [  1   7   7  94   0   0   0]
 [  2   9   1   0  46   0   0]
 [  2  16   3   0   2   0   0]
 [  0   1  11   0   0   0   6]]


## 모델 저장

In [ ]:
import joblib

joblib.dump(cnb_clf, 'clf_news_CNB_v1.pkl', compress=True)

['clf_news_CNB_v1.pkl']